### Compile Attributes
Iterates through the downloaded fundamental csv files and compiles the data for each fundamental attribute (119). CSV fiules are then stored to 'fundamentals_by_attribute' folder.

In [5]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import os
import urllib
import json
import requests

from io import StringIO
from pathlib import Path

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from eod import EodHistoricalData

pd.set_option('display.max_rows', None)

In [6]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'

In [7]:
os.chdir(path_prefix)

In [8]:
api_key = "618f834b7bfe27.18132752"
client = EodHistoricalData(api_key)

In [9]:
resp = client.get_exchange_symbols(exchange='NYSE')

In [10]:
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [ ]:
for ticker in tickers:
    filepath = Path(path_prefix + path_fun + '\df_fun_{}.csv'.format(ticker))
    print(ticker)
    if os.path.isfile(filepath) == True:
        df_fun = pd.read_csv(filepath, index_col=0).transpose()
        for i in range(len(df_fun.columns)):
            col = df_fun.columns[i]
            print(col)
            filepath = Path(path_prefix + path_att + 'df_{}.csv'.format(col))
            if os.path.isfile(filepath) == False:
                filepath.parent.mkdir(parents=True, exist_ok=True)  
                df_fun[col].to_csv(filepath)
            else:  
                df_att = pd.read_csv(filepath, index_col=0)
                att = df_fun[col].to_frame()
                suffix = '_{}'.format(ticker)
                df_att = df_att.join(att, how='outer', rsuffix=suffix)
                df_att.to_csv(filepath)